In [1]:
1+1

2

In [17]:
import pandas as pd
from datetime import datetime
from osgeo import gdal
import numpy as np
import subprocess
import glob
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
from dateutil.parser import parse
from tensorflow import keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from collections import Counter

In [18]:
train_labels = pd.read_csv("train_labels.csv")
grid_metadata = pd.read_csv("grid_metadata.csv")
satellite_metadata = pd.read_csv("pm25_satellite_metadata.csv")
satellite_metadata['Date'] =  pd.to_datetime(satellite_metadata['time_end'], format='%Y-%m-%d')

In [19]:
test_labels = pd.read_csv("submission_format.csv")

In [20]:
train_labels

,datetime,grid_id,value
0,2018-02-01T08:00:00Z,3S31A,11.400000
1,2018-02-01T08:00:00Z,A2FBI,17.000000
2,2018-02-01T08:00:00Z,DJN0F,11.100000
3,2018-02-01T08:00:00Z,E5P9N,22.100000
4,2018-02-01T08:00:00Z,FRITQ,29.800000
...,...,...,...
34307,2020-12-31T18:30:00Z,P8JA5,368.611111
34308,2020-12-31T18:30:00Z,PW0JT,294.425000
34309,2020-12-31T18:30:00Z,VXNN3,224.857143
34310,2020-12-31T18:30:00Z,VYH7U,287.000000


In [21]:
import glob
len(glob.glob("../../data/raw/train/maiac/*/*.hdf"))


all_files= glob.glob("../../data/raw/train/maiac/*/*.hdf")

In [22]:
len(all_files)

4260

In [23]:
all_files[0]

'../../data/raw/train/maiac/2019/20191222T032500_maiac_tpe_1.hdf'

In [24]:
train_labels

,datetime,grid_id,value
0,2018-02-01T08:00:00Z,3S31A,11.400000
1,2018-02-01T08:00:00Z,A2FBI,17.000000
2,2018-02-01T08:00:00Z,DJN0F,11.100000
3,2018-02-01T08:00:00Z,E5P9N,22.100000
4,2018-02-01T08:00:00Z,FRITQ,29.800000
...,...,...,...
34307,2020-12-31T18:30:00Z,P8JA5,368.611111
34308,2020-12-31T18:30:00Z,PW0JT,294.425000
34309,2020-12-31T18:30:00Z,VXNN3,224.857143
34310,2020-12-31T18:30:00Z,VYH7U,287.000000


In [25]:
ls

Untitled.ipynb                          submission_format.csv
Untitled1.ipynb                         test/
checkother.ipynb                        train/
data_exploration.ipynb                  train_labels.csv
dumbSameMonthAverage.csv                v1_base_same_month_daystuff.ipynb
grid_metadata.csv                       v2_acleaning_dding_satellte_data.ipynb
pm25_satellite_metadata.csv


In [26]:
ls ..

external/   interim/    processed/  processed2/ raw/


In [27]:
import json

In [28]:
from functools import partial
def rast_each_info(rasteri,j):
    try:
        band = rasteri.GetRasterBand(1)
        band_arr = band.ReadAsArray()
        return band_arr.tolist()
    except:
        return None


In [29]:
import orjson

In [44]:
all_data = []
def get_all_data_for_loci(ds,granule_id,parellel = False):
    each = {}
    metadata = ds.GetMetadata()
    for i in range(len(ds.GetSubDatasets())):
        raster = gdal.Open(ds.GetSubDatasets()[i][0]) #grid5km:cosSZA features only
        each_raster = raster.GetMetadata()
#         print(each_raster)
        long_name = each_raster["long_name"]
#         print(long_name)
         
        all_rasters = []
        if(parellel):
            listi = list(range(int(raster.GetMetadata()["ADDITIONALLAYERS"])))
            
            N = pool.imap(partial(rast_each_info, rasteri=raster), listi)
            
            print(N)
        else:
            for j in range(int(raster.GetMetadata()["ADDITIONALLAYERS"])):
                try:
                    band = raster.GetRasterBand(j+1)
                    band_arr = band.ReadAsArray()
                    all_rasters.append(band_arr.tolist())
                except:
                    pass
            each[long_name] = all_rasters
    
    each_data_f = {'file':granule_id,'metadata':metadata,'data':each}
    return each_data_f
#     with open("../processed2/"+granule_id +'.pickle', 'wb') as handle:
#         pickle.dump(each_data_f, handle)
#     with open("../processed2/"+granule_id + ".json", "w") as outfile:
#         orjson.dumps(each_data_f, outfile)
    

In [45]:
def get_all_data_for_loci_specific(ds,granule_id,parellel = False):
    each = {}
    metadata = ds.GetMetadata()
    
    raster = gdal.Open(ds.GetSubDatasets()[1][0]) #grid5km:cosSZA features only
    each_raster = raster.GetMetadata()
    long_name = each_raster["long_name"]

    all_rasters = []
    if(parellel):
        listi = list(range(int(raster.GetMetadata()["ADDITIONALLAYERS"])))

        N = pool.imap(partial(rast_each_info, rasteri=raster), listi)

        print(N)
    else:
        for j in range(int(raster.GetMetadata()["ADDITIONALLAYERS"])):
            try:
                band = raster.GetRasterBand(j+1)
                band_arr = band.ReadAsArray()
                all_rasters.append(band_arr.tolist())
            except:
                pass
        each[long_name] = all_rasters
    
    each_data_f = {'file':granule_id,'metadata':metadata,'data':each}
    del ds
    return each_data_f

In [46]:
# ls test/misr/2017

In [47]:
# ds = gdal.Open("test/misr/2017/20171101T192120_misr_la_0.nc")
# metadata = ds.GetMetadata()


In [48]:
# res = get_all_data_for_loci(ds,"20171101T192120_misr_la_0.nc",parellel = False)

In [49]:

# import pickle
# with open('../processed2/20180308T031500_maiac_tpe_0.hdf.pickle', 'rb') as handle:
#     b = pickle.load(handle)

In [50]:
import pickle

In [51]:
# Opens the HDF file
def load_data(FILEPATH):
    ds = gdal.Open(FILEPATH)
    return ds

def format_file_path(granule_id):
    year = granule_id[:4]
    res = '../../data/raw/train/maiac/'+year+'/'+granule_id
    return res

def fetch_subset(granule_id):
    formatted = format_file_path(granule_id)
    ds = load_data( formatted)
    return get_all_data_for_loci(ds,granule_id)

In [52]:
all_files_flatten[1]

NameError: name 'all_files_flatten' is not defined

In [53]:
%%time
res = fetch_subset('20180308T031500_maiac_tpe_0.hdf')

CPU times: user 2.15 s, sys: 430 ms, total: 2.58 s
Wall time: 2.68 s


In [82]:
res["data"].keys()

dict_keys(['AOD at 0.55 micron'])

In [46]:
# np.array(res["data"]["AOD_QA"])

In [54]:
from multiprocessing import get_context
p = get_context("fork").Pool(16)
all_files_d_100 = []
for x in tqdm.tqdm(p.imap_unordered(fetch_subset, all_files_flatten), total=len(all_files_flatten)):
    all_files_d_100.append(x)

NameError: name 'tqdm' is not defined

In [42]:
len(all_files_d_100)

20

In [62]:
with open("../processed2/AOD at 0.47 micron/100.pickle", 'wb') as handle:
        pickle.dump(all_files_d_100, handle)

In [67]:
len(all_files_flatten)

4234

In [68]:
res["data"].keys()

dict_keys(['AOD_QA'])

In [83]:
for i in range(len(all_files_flatten)//2,len(all_files_flatten),20):
    print(i,i+20)
    p = get_context("fork").Pool(10)
    all_files_d_100 = []
    for x in tqdm.tqdm(p.imap_unordered(fetch_subset, all_files_flatten[i:i+20]), total=len(all_files_flatten[i:i+20])):
        all_files_d_100.append(x)
    with open("../processed2/AOD at 0.55 micron/"+str(i+20)+".pickle", 'wb') as handle:
        pickle.dump(all_files_d_100, handle)

2117 2137


100%|██████████| 20/20 [00:10<00:00,  1.98it/s]


2137 2157


100%|██████████| 20/20 [00:08<00:00,  2.37it/s]


2157 2177


...

100%|██████████| 20/20 [00:10<00:00,  1.96it/s]


4197 4217


100%|██████████| 20/20 [00:11<00:00,  1.78it/s]


4217 4237


100%|██████████| 17/17 [00:13<00:00,  1.25it/s]


In [47]:
np.array(res["data"]['AOD at 0.47 micron']).shape

(4, 1200, 1200)

In [69]:
len(all_files_flatten)

4234

In [70]:
for i in range(0,len(all_files_flatten),100):
    print(i)
    all_files_d = []
    tasks = all_files_flatten[i:i+100]
    for x in tqdm.tqdm(p.imap_unordered(fetch_subset, tasks), total=len(tasks)):
        all_files_d.append(x)
    with open("../processed2/"+str(i) +'.pickle', 'wb') as handle:
        pickle.dump(all_files_d, handle)
    break

In [73]:
ls "../processed2/AOD at 0.55 micron"

100.pickle   1320.pickle  1640.pickle  1980.pickle  380.pickle   700.pickle
1000.pickle  1340.pickle  1660.pickle  20.pickle    40.pickle    720.pickle
1020.pickle  1360.pickle  1680.pickle  200.pickle   400.pickle   740.pickle
1040.pickle  1380.pickle  1700.pickle  2000.pickle  420.pickle   760.pickle
1060.pickle  140.pickle   1720.pickle  2020.pickle  440.pickle   780.pickle
1080.pickle  1400.pickle  1740.pickle  2040.pickle  460.pickle   80.pickle
1100.pickle  1420.pickle  1760.pickle  2060.pickle  480.pickle   800.pickle
1120.pickle  1440.pickle  1780.pickle  2080.pickle  500.pickle   820.pickle
1140.pickle  1460.pickle  180.pickle   2100.pickle  520.pickle   840.pickle
1160.pickle  1480.pickle  1800.pickle  2120.pickle  540.pickle   860.pickle
1180.pickle  1500.pickle  1820.pickle  220.pickle   560.pickle   880.pickle
120.pickle   1520.pickle  1840.pickle  240.pickle   580.pickle   900.pickle
1200.pickle  1540.pickle  1860.pickle  260.pickle   60.pickle    920.pickle
1220.pickle  

In [62]:
for i in res["data"].keys():
    print(i)

AOD at 0.47 micron
AOD at 0.55 micron
AOD uncertainty at 0.47 micron, range 0-4
Fine mode fraction for Ocean
Column Water Vapor (in cm liquid water)
AOD_QA
Regional background model used
Smoke Injection Height over local surface height, in meters
cosine of Solar Zenith Angle
cosine of View Zenith Angle
Relative Azimuth Angle
Scattering Angle
Glint Angle


In [63]:
len(all_files_flatten)

NameError: name 'all_files_flatten' is not defined

In [11]:
def get_only_specific(loc , type_d= 'AOD at 0.47 micron'):
    res = fetch_subset(loc)
    vals = np.array(res["data"][type_d])
    vals = [i.tolist() for i in vals if len(Counter(i.flatten()))!=1]
    return {"name":loc,"data":vals}

In [12]:
tasks = all_files_flatten[:100]
all_files_d = []
for x in tqdm.tqdm(p.imap_unordered(get_only_specific, tasks), total=len(tasks)):
    all_files_d.append(x)

NameError: name 'all_files_flatten' is not defined

In [13]:
from multiprocessing import get_context

p = get_context("fork").Pool(8)

In [14]:
from pymongo import MongoClient
from pprint import pprint
import gridfs
client = MongoClient("localhost:27017")
mydb = client["NASA"]
collection = db['maiac']

NameError: name 'db' is not defined

In [15]:
%%time
a = get_only_specific('20180308T031500_maiac_tpe_0.hdf')

NameError: name 'fetch_subset' is not defined

In [16]:
from sys import getsizeof
getsizeof(a)

NameError: name 'a' is not defined

In [17]:
np.array(a["data"]).nbytes

NameError: name 'a' is not defined

In [18]:
collection.insert_one(a)

NameError: name 'collection' is not defined

In [19]:
# # Let us create a dataset of AOD
# from multiprocessing import get_context

# p = get_context("fork").Pool(8)

# all_files_d = []
# for x in tqdm.tqdm(p.imap_unordered(get_ds_loc, tasks), total=len(tasks)):
#     all_files_d.append(x)

In [48]:
# Let us then convert the values to a list
vals = np.array(res["data"]['AOD at 0.47 micron'])
vals = [i for i in vals if len(Counter(i.flatten()))!=1]


NameError: name 'res' is not defined

In [49]:
vals = np.array(res["data"]['AOD at 0.47 micron'])
len(vals[0].tolist())

NameError: name 'res' is not defined

In [50]:
len(Counter(vals[2].flatten()))

NameError: name 'vals' is not defined

In [51]:
from matplotlib.pyplot import figure

figure(figsize=(10, 8), dpi=80)
plt.imshow(np.where(vals[3] <0, 0, vals[3]), cmap = "Blues")
plt.colorbar()
plt.show()

NameError: name 'vals' is not defined

<Figure size 800x640 with 0 Axes>

In [52]:
from collections import Counter
Counter(np.array(res["data"]['AOD at 0.47 micron'])[0].flatten())

NameError: name 'res' is not defined

In [53]:
from multiprocessing import get_context

p = get_context("fork").Pool(8)

for _ in tqdm.tqdm(p.imap_unordered(fetch_subset, all_files_flatten), total=len(all_files_flatten)):
    pass

NameError: name 'all_files_flatten' is not defined

In [54]:
ls

Untitled.ipynb                          submission_format.csv
Untitled1.ipynb                         test/
checkother.ipynb                        train/
data_exploration.ipynb                  train_labels.csv
dumbSameMonthAverage.csv                v1_base_same_month_daystuff.ipynb
grid_metadata.csv                       v2_acleaning_dding_satellte_data.ipynb
pm25_satellite_metadata.csv


In [55]:
1+1

2

In [56]:
all_files_flatten

NameError: name 'all_files_flatten' is not defined

In [57]:
def get_grid_data(metadata, grid_id):
    return metadata[metadata["grid_id"] == grid_id]

In [58]:
def fetch_satellite_meta(metadata, datetime, location, datatype, split):
    if location == "Delhi":
        location = "dl"
    elif location == "Taipei":
        location = "tpe"
    else:
        location = "la"
    metadata = metadata[metadata['location'] == location]
    metadata = metadata[metadata['product'] == datatype]
    metadata = metadata[metadata['split'] == split]
    dateobject = parse(datetime)
    return metadata.loc[(metadata['Date'].dt.month == dateobject.month) & 
                        (metadata['Date'].dt.day == dateobject.day) &
                        (metadata['Date'].dt.year <= dateobject.year)]

In [59]:
def fetch_training_features(grid_id, datetime, split):
    temp = get_grid_data(grid_metadata, grid_id)
    sat_met = fetch_satellite_meta(satellite_metadata, 
                               datetime, 
                               temp.iloc[0]['location'], 
                               "maiac", 
                               split)
    counter = 0
    features = None
    for i in range(len(sat_met)):
        counter+=1
        granule_id = sat_met.iloc[i]['granule_id']

        subset = fetch_subset(granule_id)
        if features is None:
            features = subset
        else:
            features+=subset
    return features/counter

In [60]:
def fetch_training_features(grid_id, datetime, split):
    temp = get_grid_data(grid_metadata, grid_id)
    sat_met = fetch_satellite_meta(satellite_metadata, 
                               datetime, 
                               temp.iloc[0]['location'], 
                               "maiac", 
                               split)
    counter = 0
    features = None
    granule_id_loc = []
    for i in range(len(sat_met)):
        counter+=1
        granule_id = sat_met.iloc[i]['granule_id']
        granule_id_loc.append(granule_id)
    return granule_id_loc
        
#         subset = fetch_subset(granule_id)
        

In [61]:
from multiprocessing import Pool
import tqdm
import time

In [62]:
from multiprocessing import Pool
import tqdm

features = []
split = "train"
pool = Pool(processes=8)


In [63]:
tasks = list(zip(train_labels["grid_id"],train_labels["datetime"]))
tasks[:5]

[('3S31A', '2018-02-01T08:00:00Z'),
 ('A2FBI', '2018-02-01T08:00:00Z'),
 ('DJN0F', '2018-02-01T08:00:00Z'),
 ('E5P9N', '2018-02-01T08:00:00Z'),
 ('FRITQ', '2018-02-01T08:00:00Z')]

In [64]:
def get_ds_loc(i):
    feature = fetch_training_features(i[0], i[1], split)
    return feature

In [65]:
get_ds_loc(tasks[5])

['20180201T191000_maiac_la_0.hdf']

In [66]:
# counter = 0
# all_filess = []
# for i in tqdm.tqdm(tasks):
#     counter +=1
#     all_filess.append(get_ds_loc(i))
# #     if(counter %100==0):
# #         print(counter)

In [67]:
from multiprocessing import get_context

p = get_context("fork").Pool(8)

all_files_d = []
for x in tqdm.tqdm(p.imap_unordered(get_ds_loc, tasks), total=len(tasks)):
    all_files_d.append(x)

100%|██████████| 34312/34312 [00:18<00:00, 1823.63it/s]


In [93]:
# print(all_files_d)

In [75]:
len(all_files_d)

34312

In [76]:
all_files_flatten = [j for i in all_files_d for j in i ]

In [77]:
all_files_flatten = list(set(all_files_flatten))

In [78]:
len(all_files_flatten)

4234

In [39]:
all_files_flatten

['20181207T192500_maiac_la_0.hdf',
 '20201119T025500_maiac_tpe_0.hdf',
 '20200213T201000_maiac_la_0.hdf',
 '20180814T055000_maiac_dl_0.hdf',
 '20180608T034000_maiac_tpe_1.hdf',
 '20200217T032000_maiac_tpe_1.hdf',
 '20191031T201500_maiac_la_0.hdf',
 '20180221T035500_maiac_tpe_0.hdf',
 '20190511T042000_maiac_tpe_0.hdf',
 '20190607T054000_maiac_dl_0.hdf',
 '20200219T030500_maiac_tpe_0.hdf',
 '20200603T192500_maiac_la_0.hdf',
 '20200601T031500_maiac_tpe_0.hdf',
 '20191021T063000_maiac_dl_0.hdf',
 '20180713T055000_maiac_dl_0.hdf',
 '20200303T041500_maiac_tpe_0.hdf',
 '20201201T194500_maiac_la_0.hdf',
 '20190713T200000_maiac_la_0.hdf',
 '20200404T041500_maiac_tpe_0.hdf',
 '20201227T071500_maiac_dl_0.hdf',
 '20190618T201000_maiac_la_0.hdf',
 '20180916T031500_maiac_tpe_0.hdf',
 '20180222T030000_maiac_tpe_0.hdf',
 '20180313T065000_maiac_dl_0.hdf',
 '20180228T022500_maiac_tpe_0.hdf',
 '20190607T202500_maiac_la_0.hdf',
 '20200906T035500_maiac_tpe_0.hdf',
 '20191010T195500_maiac_la_0.hdf',
 '20200

In [47]:
def format_file_path(granule_id):
    year = granule_id[:4]
    res = '../../data/raw/train/maiac/'+year+'/'+granule_id
    return res

In [48]:
def fetch_subset(granule_id):
    
#     print(granule_id)
    
    formatted = format_file_path(granule_id)
    ds = load_data( formatted)
    get_all_data_for_loc(ds,granule_id)

In [49]:
all_data 

[]

In [50]:
all_files

['../../data/raw/train/maiac/2019/20191222T032500_maiac_tpe_1.hdf',
 '../../data/raw/train/maiac/2019/20190120T032500_maiac_tpe_1.hdf',
 '../../data/raw/train/maiac/2019/20190728T025500_maiac_tpe_0.hdf',
 '../../data/raw/train/maiac/2019/20190320T030500_maiac_tpe_0.hdf',
 '../../data/raw/train/maiac/2019/20190917T064500_maiac_dl_0.hdf',
 '../../data/raw/train/maiac/2019/20191110T025000_maiac_tpe_0.hdf',
 '../../data/raw/train/maiac/2019/20190615T063000_maiac_dl_0.hdf',
 '../../data/raw/train/maiac/2019/20190322T025500_maiac_tpe_1.hdf',
 '../../data/raw/train/maiac/2019/20190422T021000_maiac_tpe_0.hdf',
 '../../data/raw/train/maiac/2019/20190408T033500_maiac_tpe_0.hdf',
 '../../data/raw/train/maiac/2019/20191026T033000_maiac_tpe_1.hdf',
 '../../data/raw/train/maiac/2019/20190720T020500_maiac_tpe_0.hdf',
 '../../data/raw/train/maiac/2019/20191115T030500_maiac_tpe_1.hdf',
 '../../data/raw/train/maiac/2019/20191117T025500_maiac_tpe_0.hdf',
 '../../data/raw/train/maiac/2019/20190328T035500_

In [57]:
import multiprocessing

multiprocessing.get_start_method()

'spawn'

In [58]:
from multiprocessing import get_context

p = get_context("fork").Pool(8)

Process ForkPoolWorker-47:
Process ForkPoolWorker-44:
Process ForkPoolWorker-46:
Process ForkPoolWorker-48:
Process ForkPoolWorker-41:
Process ForkPoolWorker-45:
Process ForkPoolWorker-43:
Process ForkPoolWorker-42:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/homebrew/anaconda3/envs/cv/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/opt/homebrew/anaconda3/envs/cv/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/opt/homebrew/anaconda3/envs/cv/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/opt/homebrew/anaconda3/envs/cv/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/opt/hom

In [59]:
len(all_files_flatten)

4234

In [60]:
# from ray.util.multiprocessing import Pool

In [57]:
# pool = Pool(processes=4)
# for _ in tqdm.tqdm(pool.imap_unordered(fetch_subset, all_files_flatten), total=len(all_files_flatten)):
#     pass

In [35]:
for i in tqdm.tqdm(all_files_flatten):
    fetch_subset(i)

  3%|▎         | 122/4234 [48:57<27:30:24, 24.08s/it]


KeyboardInterrupt: 

In [62]:
len(all_data)

2

In [43]:
def fetch_subset(granule_id):
    
    print(granule_id)
    
    formatted = format_file_path(granule_id)
    ds = load_data( formatted)
    get_all_data_for_loc(ds)

In [21]:
# pool.map(get_ds_loc, tasks[:10])

In [23]:
# features = []
# for x in tqdm.tqdm(pool.imap_unordered(get_ds_loc, tasks[:10]), total=len(tasks[:10])):
#     features.append(x)

In [51]:
features = []
split = "train"
for i in range(len(train_labels)):
    print(i)
    feature = fetch_training_features(train_labels.iloc[i]['grid_id'], train_labels.iloc[i]['datetime'], split)
    features.append(feature)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108


3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708
3709
3710
3711
3712
3713
3714
3715
3716
3717
3718
3719
3720
3721
3722
3723
3724
3725
3726
3727
3728
3729
3730
3731
3732
3733
3734
3735
3736
3737
3738
3739
3740
3741
3742
3743
3744
3745
3746
3747
3748
3749
3750
3751
3752
3753
3754
3755
3756
3757
3758
3759
3760
3761
3762
3763
3764
3765
3766
3767
3768
3769
3770
3771
3772
3773
3774
3775
3776
3777


5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342
5343
5344
5345
5346
5347
5348
5349
5350
5351
5352
5353
5354
5355
5356
5357
5358
5359
5360
5361
5362
5363
5364
5365
5366
5367
5368
5369
5370
5371
5372
5373
5374
5375
5376
5377
5378
5379
5380
5381
5382
5383
5384
5385
5386
5387
5388
5389
5390
5391
5392
5393
5394
5395
5396
5397
5398
5399
5400
5401
5402
5403
5404
5405
5406
5407
5408
5409
5410
5411
5412
5413
5414
5415
5416
5417
5418
5419
5420
5421
5422
5423
5424
5425
5426
5427
5428
5429
5430
5431
5432
5433


6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977
6978
6979
6980
6981
6982
6983
6984
6985
6986
6987
6988
6989
6990
6991
6992
6993
6994
6995
6996
6997
6998
6999
7000
7001
7002
7003
7004
7005
7006
7007
7008
7009
7010
7011
7012
7013
7014
7015
7016
7017
7018
7019
7020
7021
7022
7023
7024
7025
7026
7027
7028
7029
7030
7031
7032
7033
7034
7035
7036
7037
7038
7039
7040
7041
7042
7043
7044
7045
7046
7047
7048
7049
7050
7051
7052
7053
7054
7055
7056
7057
7058
7059
7060
7061
7062
7063
7064
7065
7066
7067
7068
7069
7070
7071
7072
7073
7074
7075
7076
7077
7078
7079
7080
7081
7082
7083
7084
7085
7086
7087
7088
7089
7090
7091
7092
7093
7094
7095
7096
7097
7098
7099
7100
7101
7102
7103
7104
7105
7106
7107
7108
7109
7110
7111
7112
7113
7114
7115
7116
7117
7118
7119
7120
7121
7122
7123
7124
7125
7126
7127
7128
7129
7130
7131
7132
7133
7134


8580
8581
8582
8583
8584
8585
8586
8587
8588
8589
8590
8591
8592
8593
8594
8595
8596
8597
8598
8599
8600
8601
8602
8603
8604
8605
8606
8607
8608
8609
8610
8611
8612
8613
8614
8615
8616
8617
8618
8619
8620
8621
8622
8623
8624
8625
8626
8627
8628
8629
8630
8631
8632
8633
8634
8635
8636
8637
8638
8639
8640
8641
8642
8643
8644
8645
8646
8647
8648
8649
8650
8651
8652
8653
8654
8655
8656
8657
8658
8659
8660
8661
8662
8663
8664
8665
8666
8667
8668
8669
8670
8671
8672
8673
8674
8675
8676
8677
8678
8679
8680
8681
8682
8683
8684
8685
8686
8687
8688
8689
8690
8691
8692
8693
8694
8695
8696
8697
8698
8699
8700
8701
8702
8703
8704
8705
8706
8707
8708
8709
8710
8711
8712
8713
8714
8715
8716
8717
8718
8719
8720
8721
8722
8723
8724
8725
8726
8727
8728
8729
8730
8731
8732
8733
8734
8735
8736
8737
8738
8739
8740
8741
8742
8743
8744
8745
8746
8747
8748
8749
8750
8751
8752
8753
8754
8755
8756
8757
8758
8759
8760
8761
8762
8763
8764
8765
8766
8767
8768
8769
8770
8771
8772
8773
8774
8775
8776
8777
8778
8779


KeyboardInterrupt: 

In [49]:
features

[['20180201T191000_maiac_la_0.hdf'],
 ['20180201T191000_maiac_la_0.hdf'],
 ['20180201T191000_maiac_la_0.hdf'],
 ['20180201T191000_maiac_la_0.hdf'],
 ['20180201T191000_maiac_la_0.hdf'],
 ['20180201T191000_maiac_la_0.hdf'],
 ['20180201T191000_maiac_la_0.hdf'],
 ['20180201T191000_maiac_la_0.hdf'],
 ['20180201T191000_maiac_la_0.hdf'],
 ['20180201T191000_maiac_la_0.hdf']]

In [37]:
len(all_data),len(train_labels)

(702, 34312)

In [38]:
def generate_features(train_labels, split):
    labels = []
    features = []
    for i in range(len(train_labels[:10])):
        feature = fetch_training_features(train_labels.iloc[i]['grid_id'], train_labels.iloc[i]['datetime'], split)
        features.append(np.array(feature).reshape(-1))
        if split == "train":
            labels.append(train_labels.iloc[i]['value'])
    return np.array(features), np.array(labels)

In [39]:
train_labels[:10]

,datetime,grid_id,value
0,2018-02-01T08:00:00Z,3S31A,11.4
1,2018-02-01T08:00:00Z,A2FBI,17.0
2,2018-02-01T08:00:00Z,DJN0F,11.1
3,2018-02-01T08:00:00Z,E5P9N,22.1
4,2018-02-01T08:00:00Z,FRITQ,29.8
5,2018-02-01T08:00:00Z,H96P6,12.8
6,2018-02-01T08:00:00Z,PG3MI,21.0
7,2018-02-01T08:00:00Z,QJHW4,5.2
8,2018-02-01T08:00:00Z,WT52R,6.5
9,2018-02-01T08:00:00Z,X5DKW,17.7


In [40]:
train_labels

,datetime,grid_id,value
0,2018-02-01T08:00:00Z,3S31A,11.400000
1,2018-02-01T08:00:00Z,A2FBI,17.000000
2,2018-02-01T08:00:00Z,DJN0F,11.100000
3,2018-02-01T08:00:00Z,E5P9N,22.100000
4,2018-02-01T08:00:00Z,FRITQ,29.800000
...,...,...,...
34307,2020-12-31T18:30:00Z,P8JA5,368.611111
34308,2020-12-31T18:30:00Z,PW0JT,294.425000
34309,2020-12-31T18:30:00Z,VXNN3,224.857143
34310,2020-12-31T18:30:00Z,VYH7U,287.000000


In [41]:
features, labels = generate_features(train_labels, "train")

1
20180201T191000_maiac_la_0.hdf
1
20180201T191000_maiac_la_0.hdf
1
20180201T191000_maiac_la_0.hdf
1
20180201T191000_maiac_la_0.hdf
1
20180201T191000_maiac_la_0.hdf
1
20180201T191000_maiac_la_0.hdf
1
20180201T191000_maiac_la_0.hdf
1
20180201T191000_maiac_la_0.hdf
1
20180201T191000_maiac_la_0.hdf
1
20180201T191000_maiac_la_0.hdf


In [42]:
fetch_subset(all_files[0])

../../data/raw/train/maiac/2019/20191222T032500_maiac_tpe_1.hdf


AttributeError: 'NoneType' object has no attribute 'GetMetadata'

In [43]:
test_labels

,datetime,grid_id,value
0,2017-01-07T16:00:00Z,1X116,0.0
1,2017-01-07T16:00:00Z,9Q6TA,0.0
2,2017-01-07T16:00:00Z,KW43U,0.0
3,2017-01-07T16:00:00Z,VR4WG,0.0
4,2017-01-07T16:00:00Z,XJF9O,0.0
...,...,...,...
13499,2021-08-24T08:00:00Z,QJHW4,0.0
13500,2021-08-24T08:00:00Z,VBLD0,0.0
13501,2021-08-24T08:00:00Z,WT52R,0.0
13502,2021-08-24T08:00:00Z,ZP1FZ,0.0


In [10]:
# Let us do this for one only|

In [16]:
loc = "1X116"
one_train = train_labels[train_labels["grid_id"]==loc]
one_test = test_labels[test_labels["grid_id"]==loc]
one_train.head()

,datetime,grid_id,value
12,2018-02-01T16:00:00Z,1X116,12.857143
31,2018-02-02T16:00:00Z,1X116,27.062500
61,2018-02-03T16:00:00Z,1X116,8.125000
91,2018-02-04T16:00:00Z,1X116,13.300000
122,2018-02-05T16:00:00Z,1X116,22.000000


In [18]:
one_test.head()

,datetime,grid_id,value
0,2017-01-07T16:00:00Z,1X116,0.0
17,2017-01-08T16:00:00Z,1X116,0.0
34,2017-01-09T16:00:00Z,1X116,0.0
51,2017-01-10T16:00:00Z,1X116,0.0
56,2017-01-11T16:00:00Z,1X116,0.0


In [19]:
col = one_train["datetime"]

In [62]:
t_final = []


for loc in list(set(train_labels["grid_id"])):
    print(loc)
    one_train = train_labels[train_labels["grid_id"]==loc]
    one_test = test_labels[test_labels["grid_id"]==loc]
    col = one_train["datetime"]
    for i in one_test["datetime"]:
        strr_search = i[5:]
#         print(strr_search)
        vals = one_train[col.str.contains(strr_search)]["value"]
#         print(vals)
        mean_pm25 = vals.mean()
        if(mean_pm25!=mean_pm25):
            mean_pm25 = t_final[-1][2]
        t_final.append((i,loc,mean_pm25))

E2AUK
HANW9
HM74A
6EIL6
PG3MI
7F1D1
IUMEZ
QJHW4
90S79
WZNCR
AZJ0Z
A7UCQ
FRITQ
78V83
ZP1FZ
9Q6TA
VBLD0
H96P6
SZLMT
C7PGV
KW43U
S77YN
VXNN3
DJN0F
PW0JT
90BZ1
ZF3ZW
XNLVD
GAC6R
UC74Z
GJLB2
PJNW1
3S31A
D7S1G
NE7BV
YHOPV
VR4WG
CPR0W
KZ9W9
E5P9N
WT52R
D72OT
XJF9O
DHO4M
P8JA5
GVQXS
ZZ8JF
VYH7U
X5DKW
A2FBI
7334C
1X116
1Z2W7
8KNI6


In [66]:
test_cal = pd.DataFrame(t_final, columns =["datetime","grid_id","calc_value"]) 
test_cal.head()

,datetime,grid_id,calc_value
0,2017-11-27T18:30:00Z,E2AUK,61.417510
1,2017-11-28T18:30:00Z,E2AUK,71.205725
2,2017-11-29T18:30:00Z,E2AUK,104.593217
3,2017-11-30T18:30:00Z,E2AUK,166.150314
4,2017-12-01T18:30:00Z,E2AUK,156.985630


In [59]:
test_labels

,datetime,grid_id,value
0,2017-01-07T16:00:00Z,1X116,0.0
1,2017-01-07T16:00:00Z,9Q6TA,0.0
2,2017-01-07T16:00:00Z,KW43U,0.0
3,2017-01-07T16:00:00Z,VR4WG,0.0
4,2017-01-07T16:00:00Z,XJF9O,0.0
...,...,...,...
13499,2021-08-24T08:00:00Z,QJHW4,0.0
13500,2021-08-24T08:00:00Z,VBLD0,0.0
13501,2021-08-24T08:00:00Z,WT52R,0.0
13502,2021-08-24T08:00:00Z,ZP1FZ,0.0


In [68]:
final = pd.merge(test_labels,test_cal,on = ["datetime","grid_id"]).drop(columns = ["value"])

In [69]:
final.head()

,datetime,grid_id,calc_value
0,2017-01-07T16:00:00Z,1X116,12.719368
1,2017-01-07T16:00:00Z,9Q6TA,14.291667
2,2017-01-07T16:00:00Z,KW43U,12.565217
3,2017-01-07T16:00:00Z,VR4WG,15.853261
4,2017-01-07T16:00:00Z,XJF9O,14.264302


In [70]:
final.columns = ["datetime","grid_id","value"]

In [73]:
final.to_csv("dumbSameMonthAverage.csv",index=False)